In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [58]:
os.listdir()

['.ipynb_checkpoints',
 'austin_extact_code.ipynb',
 'boston_extract_code.ipynb',
 'chattanooga_extract_code.ipynb',
 'chicago_extract_code.ipynb',
 'creds',
 'kc_extract_code.ipynb',
 'littlerock_extract_code.ipynb',
 'nyc_extract_code.ipynb',
 'SF_extract_code.ipynb']

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



gc = gspread.authorize(credentials)

wks = gc.open("NYTimes Mini Crossword times").sheet1


data = wks.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)


df = df.replace(np.NaN, -1)

# saving a data frame to a buffer (same as with a regular file):
df.to_csv('nytimescrossword.csv',index=False)

# NYC

#Initial Load
nyc_table_id = "opendatadbt.311.nyc311"
nyc311 = requests.get("https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=10000&$where=created_date >= '2021-03-18T00:00:00.000' OR closed_date >= '2021-03-18T00:00:00.000'" )
nyc311json = nyc311.json()
nyc_df = pd.DataFrame(nyc311json)
#nyc_df = nyc_df[['created_date','closed_date','agency','agency_name','descriptor','incident_zip','incident_address','city','status','latitude','longitude']]

In [60]:
#Incremental load
nyc_table_id = "opendatadbt.311.nyc311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(created_date), max(closed_date)
FROM `opendatadbt.311.nyc311`
limit 10
"""

nycmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
nycmaxopendatedate = str(nycmaxopendatedataframe['f0_'][0])

nycmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
nycmaxclosedate = str(nycmaxclosedatedataframe['f1_'][0])


In [61]:
nycmaxopendatedate

'2021-04-02T12:00:00.000'

In [62]:

nyc311 = requests.get("https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=50000&$where=created_date>'"+nycmaxopendatedate+"' OR closed_date between '" +nycmaxclosedate+ "' AND '" + current_datetime + "'") 
nyc311json = nyc311.json()
cols = ['unique_key','created_date','closed_date','agency','agency_name','descriptor','incident_zip','incident_address','city','status','latitude','longitude']
nyc_df = pd.DataFrame(nyc311json, columns=cols)
nyc_df

,unique_key,created_date,closed_date,agency,agency_name,descriptor,incident_zip,incident_address,city,status,latitude,longitude
0,46221397,2020-05-15T22:21:48.000,2021-04-03T13:18:24.000,HPD,Department of Housing Preservation and Develop...,SLOW LEAK,11237,284 COOPER STREET,BROOKLYN,Closed,40.69168317223365,-73.9034961506383
1,48245823,2020-11-24T00:15:55.000,2021-04-03T18:49:12.000,HPD,Department of Housing Preservation and Develop...,PESTS,10024,103 WEST 77 STREET,NEW YORK,Closed,40.780854415776346,-73.97688345812512
2,49514964,2021-01-11T08:31:59.000,2021-04-03T12:17:01.000,HPD,Department of Housing Preservation and Develop...,WINDOW FRAME,10462,2196 MATTHEWS AVENUE,BRONX,Closed,40.85636202841177,-73.86286032403363
3,49517644,2021-01-11T08:31:59.000,2021-04-03T12:17:01.000,HPD,Department of Housing Preservation and Develop...,DOOR,10462,2196 MATTHEWS AVENUE,BRONX,Closed,40.85636202841177,-73.86286032403363
4,49523278,2021-01-12T14:42:11.000,2021-04-03T18:33:10.000,HPD,Department of Housing Preservation and Develop...,DAMP SPOT,10033,540 FT WASHINGTON AVENUE,NEW YORK,Closed,40.85363516096966,-73.93734497240956
...,...,...,...,...,...,...,...,...,...,...,...,...
10011,50135345,2021-03-28T19:28:00.000,2021-04-03T20:10:00.000,DEP,Department of Environmental Protection,Dirty Water (WE),11427,219-50 PECK AVENUE,Queens Village,Closed,40.732499814869314,-73.7498351762619
10012,50129939,2021-03-27T08:51:00.000,2021-04-03T14:00:00.000,DEP,Department of Environmental Protection,Noise: Construction Equipment (NC1),11102,30-85 VERNON BOULEVARD,Astoria,Closed,40.76987856811654,-73.93483162139626
10013,50146560,2021-03-29T15:33:00.000,2021-04-03T21:38:00.000,DEP,Department of Environmental Protection,Dirty Water (WE),11379,73-25 68 ROAD,Middle Village,Closed,40.70988729083344,-73.87829661405529
10014,48505573,2020-12-27T21:27:45.000,2021-04-03T18:42:31.000,HPD,Department of Housing Preservation and Develop...,BELL/BUZZER/INTERCOM,10034,9 POST AVENUE,NEW YORK,Closed,40.86280316784287,-73.92474792373622


In [63]:
nyc_table_id = "opendatadbt.311.nyc311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('nyc311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    nyc_df, table_ref, job_config=job_config
).result()  # Make an API request.


In [3]:
nyc_table_id = "opendatadbt.311.nyc311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
count(*), current_date(), "NYC"
FROM `opendatadbt.311.nyc311`

"""

nycdatacount = (
    client.query(query_string)
    .result()
    .to_dataframe()
)

nycdatacount = nycdatacount.rename(columns={'f0_':'count','f1_':'date','f2_':'city'})

nyc_table_id = "opendatadbt.311.extractrowcount"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('extractrowcount')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

client.load_table_from_dataframe(
    nycdatacount, table_ref, job_config=job_config
).result()  # Make an API request.